In [1]:
# import libraries
import os
from tqdm import tqdm
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# model tools
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# model
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator


# code copied from 
from keras.utils.data_utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.keras import balanced_batch_generator

Using TensorFlow backend.


[Code Source](https://medium.com/analytics-vidhya/how-to-apply-data-augmentation-to-deal-with-unbalanced-datasets-in-20-lines-of-code-ada8521320c9)

In [2]:
base_dir = '/Users/carlostavarez/Desktop/severstal-steel-defect-detection'

In [3]:
images_tr_dir = base_dir + '/train_images'

In [4]:
df = pd.read_csv(base_dir + '/train.csv')

In [5]:
df.shape

(7095, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7095 entries, 0 to 7094
Data columns (total 3 columns):
ImageId          7095 non-null object
ClassId          7095 non-null int64
EncodedPixels    7095 non-null object
dtypes: int64(1), object(2)
memory usage: 166.4+ KB


In [7]:
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [8]:
df.tail()

,ImageId,ClassId,EncodedPixels
7090,ffcf72ecf.jpg,3,121911 34 122167 101 122422 169 122678 203 122...
7091,fff02e9c5.jpg,3,207523 3 207777 9 208030 15 208283 22 208537 2...
7092,fffe98443.jpg,3,105929 5 106177 14 106424 24 106672 33 106923 ...
7093,ffff4eaa8.jpg,3,16899 7 17155 20 17411 34 17667 47 17923 60 18...
7094,ffffd67df.jpg,3,30931 43 31103 127 31275 211 31489 253 31745 2...


In [8]:
df['ImageId'].values

array(['0002cc93b.jpg', '0007a71bf.jpg', '000a4bcdd.jpg', ...,
       'fffe98443.jpg', 'ffff4eaa8.jpg', 'ffffd67df.jpg'], dtype=object)

In [9]:
df['ClassId'].values

array([1, 3, 1, ..., 3, 3, 3])

In [17]:
x_train, x_test, y_train, y_test = train_test_split(df['ImageId'].values, df['ClassId'].values, 
                                                    random_state=42, stratify=df['ClassId'].values, train_size=0.9)

In [18]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                    random_state=42, stratify=y_train, train_size=0.9)

In [11]:
new_dir = '/Users/carlostavarez/Desktop/imgs_gen'

In [22]:
os.mkdir(new_dir)

In [12]:
train_dir = new_dir + '/train'
valid_dir = new_dir + '/valid'
test_dir = new_dir + '/test'

In [24]:
os.mkdir(train_dir)
os.mkdir(valid_dir)
os.mkdir(test_dir)

In [25]:
os.mkdir(train_dir + '/0')
os.mkdir(train_dir + '/1')
os.mkdir(train_dir + '/2')
os.mkdir(train_dir + '/3')

In [26]:
# val
os.mkdir(valid_dir + '/0')
os.mkdir(valid_dir + '/1')
os.mkdir(valid_dir + '/2')
os.mkdir(valid_dir + '/3')

# test
os.mkdir(test_dir + '/0')
os.mkdir(test_dir + '/1')
os.mkdir(test_dir + '/2')
os.mkdir(test_dir + '/3')

In [13]:
train_dir

'/Users/carlostavarez/Desktop/imgs_gen/train'

In [14]:
valid_dir

'/Users/carlostavarez/Desktop/imgs_gen/valid'

In [15]:
test_dir

'/Users/carlostavarez/Desktop/imgs_gen/test'

In [30]:
x_train

array(['cd8c479c1.jpg', '2490c490c.jpg', '12d38a1d4.jpg', ...,
       'd21be4a4b.jpg', '4a662783b.jpg', '0d9d2aa59.jpg'], dtype=object)

In [37]:
np.unique(y_train)

array([1, 2, 3, 4])

In [38]:
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1

In [39]:
np.unique(y_train)

array([0, 1, 2, 3])

In [42]:
def move_file(from_dir, to_dir, imgs_list, labels_list):
    
    for img, lbl in zip(imgs_list, labels_list):
        
        dst = to_dir +'/' + str(lbl)
        
        src = os.path.join(from_dir, img)
        dst = os.path.join(dst, img)
        shutil.copyfile(src, dst)
        
    print("{} images were copied from \n{} \nto \n{}".format(len(imgs_list), from_dir, to_dir))

In [41]:
move_file(images_tr_dir, train_dir, x_train, y_train)

5746 images were copied from /Users/carlostavarez/Desktop/severstal-steel-defect-detection/train_images to /Users/carlostavarez/Desktop/imgs_gen/train


In [43]:
move_file(images_tr_dir, valid_dir, x_val, y_val)

639 images were copied from 
/Users/carlostavarez/Desktop/severstal-steel-defect-detection/train_images 
to 
/Users/carlostavarez/Desktop/imgs_gen/valid


In [44]:
move_file(images_tr_dir, test_dir, x_test, y_test)

710 images were copied from 
/Users/carlostavarez/Desktop/severstal-steel-defect-detection/train_images 
to 
/Users/carlostavarez/Desktop/imgs_gen/test


In [7]:
def get_imgs(df):
    
    labels_list = []
    images_list = []
    
    for idx, row in df.iterrows():
        
        img_path = images_tr_dir + '/' + row['ImageId']
        label = row['ClassId']
        
        img = image.load_img(img_path, target_size=(224, 224))
        img = image.img_to_array(img)
        
        labels_list.append(label)
        images_list.append(img)
        
    return np.stack(images_list, axis=0), labels_list

In [8]:
images_list, labels_list = get_imgs(df)

In [9]:
x_trn, x_val, y_trn, y_val = train_test_split(images_list, labels_list, 
                                              stratify=labels_list, random_state=42, train_size=0.9)

In [10]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

/Users/carlostavarez/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [11]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [12]:
# resnet.layers = True

bolck_bol = False

for idx in range(len(resnet.layers)):
    
    if resnet.layers[idx].name == 'res5c_branch2b':
        bolck_bol = True
        
    if bolck_bol:
        resnet.layers[idx].trainable = True
        
    else:
        resnet.layers[idx].trainable = False

In [44]:
model = Sequential()

model.add(resnet)
model.add(Conv2D(4096, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 4096)        75501568  
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 4096)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               8389120   
_________________________________________________________________
dense_8 (Dense)              (None, 200)               102600    
_________________________________________________________________
dense_9 (Dense)              (None, 4)                

In [46]:
datagen = ImageDataGenerator(rescale=1.0/255)

In [47]:
class BalancedDataGenerator(Sequence):
    """ImageDataGenerator + RandomOversampling"""
    def __init__(self, x, y, datagen, batch_size=32):
        self.datagen = datagen
        self.batch_size = batch_size
        self._shape = x.shape        
        datagen.fit(x)
        self.gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1), y, sampler=RandomOverSampler(), batch_size=self.batch_size, keep_sparse=True)

    def __len__(self):
        return self._shape[0] // self.batch_size

    def __getitem__(self, idx):
        x_batch, y_batch = self.gen.__next__()
        x_batch = x_batch.reshape(-1, *self._shape[1:])
        return self.datagen.flow(x_batch, y_batch, batch_size=self.batch_size).next()

In [17]:
np.unique(y_trn), np.unique(y_val)

(array([1, 2, 3, 4]), array([1, 2, 3, 4]))

In [18]:
y_trn = np.array(y_trn) - 1
y_val = np.array(y_val) - 1

In [19]:
np.unique(y_trn), np.unique(y_val)

(array([0, 1, 2, 3]), array([0, 1, 2, 3]))

In [20]:
y_trn = to_categorical(y_trn)
y_val = to_categorical(y_val)

In [22]:
y_trn.shape, x_trn.shape

((6385, 4), (6385, 224, 224, 3))

In [36]:
# train_gen = datagen.flow(x_trn, y_trn, batch_size=20)
# valid_gen = datagen.flow(x_val, y_val, batch_size=20)

s_datagen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=40, 
                               width_shift_range=0.2, 
                               height_shift_range=0.2, 
                               shear_range=0.2, 
                               zoom_range=0.2,
                               horizontal_flip=True)

bgen = BalancedDataGenerator(x_trn, y_trn, s_datagen, batch_size=32)
steps_per_epoch = bgen.steps_per_epoch

In [37]:
# bgen_vl = datagen.flow(x_val, y_val, batch_size=32)
# steps_val = bgen_vl.steps_per_epoch

In [38]:
valid_gen = datagen.flow(x_val, y_val, batch_size=32)

In [39]:
steps_per_epoch, steps_val

(579, 64)

In [48]:
history = model.fit_generator(bgen, 
                              steps_per_epoch=steps_per_epoch, 
                              validation_data=valid_gen, 
                              validation_steps=len(x_val)//32, 
                              epochs=10)

Epoch 1/10
579/579 [==============================] - 3569s 6s/step - loss: 0.7329 - acc: 0.7209 - val_loss: 1.0879 - val_acc: 0.6974
Epoch 2/10
579/579 [==============================] - 3551s 6s/step - loss: 0.5215 - acc: 0.8031 - val_loss: 2.2688 - val_acc: 0.1298
Epoch 3/10
579/579 [==============================] - 3627s 6s/step - loss: 0.4656 - acc: 0.8258 - val_loss: 2.5201 - val_acc: 0.1239
Epoch 4/10
579/579 [==============================] - 3881s 7s/step - loss: 0.4254 - acc: 0.8437 - val_loss: 2.1979 - val_acc: 0.0369
Epoch 5/10
579/579 [==============================] - 3731s 6s/step - loss: 0.4047 - acc: 0.8522 - val_loss: 2.0635 - val_acc: 0.0339
Epoch 6/10
579/579 [==============================] - 3626s 6s/step - loss: 0.3921 - acc: 0.8556 - val_loss: 1.9155 - val_acc: 0.0310
Epoch 7/10
579/579 [==============================] - 3642s 6s/step - loss: 0.3725 - acc: 0.8631 - val_loss: 2.1238 - val_acc: 0.0428
Epoch 8/10
579/579 [==============================] - 3636s 6s

In [49]:
model.evaluate_generator(bgen)

[2.2005739212036133, 0.25015702843666077]

In [50]:
model.evaluate_generator(valid_gen)

[3.1303462982177734, 0.035211268812417984]

In [32]:
y_trn

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [33]:
y_trn_pred = model.predict(x_trn)

In [34]:
y_val_pred = model.predict(x_val)

In [35]:
y_trn_pred

array([[4.4343080e-03, 1.4488957e-03, 9.8969406e-01, 4.4227676e-03],
       [1.7619923e-06, 1.0551557e-07, 9.9999809e-01, 1.4338494e-08],
       [2.9367459e-04, 5.9036534e-05, 9.9931002e-01, 3.3724587e-04],
       ...,
       [5.7238364e-04, 1.6210042e-04, 9.9840242e-01, 8.6316268e-04],
       [8.1247366e-05, 1.8141752e-04, 9.9636590e-01, 3.3714673e-03],
       [1.9814749e-04, 2.8437789e-05, 9.9947459e-01, 2.9880885e-04]],
      dtype=float32)

In [43]:
p = list(y_trn_pred[0])

In [47]:
p_mx = max(p)

In [48]:
p.index(p_mx)

2

In [49]:
def convert_class(y):
    
    lb_list = []
    
    for i in y:
        
        i = list(i)
        
        i_mx = max(i)
        
        i_id = i.index(i_mx)
        
        lb_list.append(i_id)
        
    return np.array(lb_list)

In [51]:
y_tr_lb = convert_class(y_trn)

In [52]:
y_tr_lb_pred = convert_class(y_trn_pred)

In [53]:
y_vl_lb = convert_class(y_val)
y_vl_lb_pred = convert_class(y_val_pred)

In [57]:
y_trn

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [65]:
# training

confusion_matrix(y_tr_lb, y_tr_lb_pred)

array([[ 132,    4,  671,    0],
       [   2,   16,  204,    0],
       [   6,    0, 4629,    0],
       [   0,    0,  721,    0]])

In [67]:
print(classification_report(y_tr_lb, y_tr_lb_pred))

              precision    recall  f1-score   support

           0       0.94      0.16      0.28       807
           1       0.80      0.07      0.13       222
           2       0.74      1.00      0.85      4635
           3       0.00      0.00      0.00       721

    accuracy                           0.75      6385
   macro avg       0.62      0.31      0.32      6385
weighted avg       0.69      0.75      0.66      6385



In [71]:
accuracy_score(y_tr_lb, y_tr_lb_pred)

0.748159749412686

In [68]:
# validation

confusion_matrix(y_vl_lb, y_vl_lb_pred)

array([[ 16,   1,  73,   0],
       [  0,   4,  21,   0],
       [  0,   0, 515,   0],
       [  0,   0,  80,   0]])

In [69]:
print(classification_report(y_vl_lb, y_vl_lb_pred))

              precision    recall  f1-score   support

           0       1.00      0.18      0.30        90
           1       0.80      0.16      0.27        25
           2       0.75      1.00      0.86       515
           3       0.00      0.00      0.00        80

    accuracy                           0.75       710
   macro avg       0.64      0.33      0.36       710
weighted avg       0.70      0.75      0.67       710



In [72]:
accuracy_score(y_vl_lb, y_vl_lb_pred)

0.7535211267605634

## Second model

In [73]:
model = Sequential()

model.add(resnet)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

In [74]:
history = model.fit_generator(train_gen, 
                              steps_per_epoch=len(x_trn)//20, 
                              validation_data=valid_gen, 
                              validation_steps=len(x_val)//20, 
                              epochs=10)

Epoch 1/10
319/319 [==============================] - 546s 2s/step - loss: 0.5471 - acc: 0.7884 - val_loss: 1.1094 - val_acc: 0.7214
Epoch 2/10
319/319 [==============================] - 466s 1s/step - loss: 0.2994 - acc: 0.8742 - val_loss: 0.8498 - val_acc: 0.7304
Epoch 3/10
179/319 [===============>..............] - ETA: 2:59 - loss: 0.2055 - acc: 0.9128

KeyboardInterrupt: 